In [1]:
from typing_extensions import TypedDict

class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [2]:
from langchain.utilities import SQLDatabase

db_user = "root"
db_password = "root123"
db_host = "localhost"
db_name = "atliq_tshirts"

try:
    # Attempt to create a database connection
    db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)
    print("DATABASE CONNECTED SUCCESSFULLY")
except Exception as e:
    # Print failure message and the specific error
    print("DATABASE CONNECTION FAILED")
    print(f"Error: {str(e)}")
    exit(1)

DATABASE CONNECTED SUCCESSFULLY


In [3]:
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [4]:
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM t_shirts LIMIT 10;"))

mysql
['discounts', 't_shirts']
[(1, 'Levi', 'Blue', 'M', 21, 82), (2, 'Van Huesen', 'Blue', 'XL', 15, 71), (3, 'Adidas', 'White', 'M', 46, 88), (4, 'Nike', 'White', 'S', 49, 84), (5, 'Van Huesen', 'White', 'XS', 47, 53), (6, 'Nike', 'Black', 'L', 14, 65), (7, 'Nike', 'White', 'XS', 49, 48), (8, 'Levi', 'Red', 'L', 26, 78), (9, 'Nike', 'Black', 'XS', 30, 45), (10, 'Levi', 'Red', 'M', 41, 51)]


### **SETTING UP THE API  KEY**

In [5]:
from langchain import hub

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

assert len(query_prompt_template.messages) == 1

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
query_prompt_template.messages[0].pretty_print()

================================ System Message ================================

Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}

Question: {input}


In [8]:
from typing_extensions import Annotated


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [9]:
query = "How many t-shirts do we have left for Nike in extra small and white color?"
sql = write_query({"question": query})

In [10]:
print(sql.get("query"))

SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND size = 'XS' AND color = 'White' LIMIT 10;


In [11]:
from test_util import generate_sql_query

generate_sql_query("How many t-shirts do we have left for Nike in extra small and white color?", db=db)['query']


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


"SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' LIMIT 10;"

In [44]:
import ast
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool

In [43]:
import ast
def query_sql(question, sql: str) -> str:
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    answer = execute_query_tool.invoke(sql)
    parsed_result = ast.literal_eval(answer)

    # Access the first value (48)
    value = parsed_result[0][0]
    
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {question}\n'
        f'SQL Query: {sql}\n'
        f'SQL Result: {value}'
    )
    response = llm.invoke(prompt)
    return response.content
    

In [42]:
query_sql("How many t-shirts do we have left for Nike in extra small and white color?", sql)

'The user question is asking about the remaining stock of Nike t-shirts in extra small size and white color. The provided SQL query retrieves the stock quantity from the "t_shirts" table where the brand is "Nike", the size is "XS", and the color is "White". The SQL result shows that the stock quantity for this specific item is 48.\n\nTherefore, the answer to the user\'s question is: **There are 48 Nike t-shirts left in extra small and white color.**\n\nThis conclusion is based on the direct output from the SQL query executed on the database, which confirmed the available quantity for the specified criteria.'